In [87]:
#import all libraries

import urllib.request
from bs4 import BeautifulSoup
import requests
import selenium
import pandas as pd
from selenium import webdriver
import time
import re 

from selenium.common.exceptions import StaleElementReferenceException, NoSuchElementException

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

In [88]:
#lets first connect to web driver
driver= webdriver.Chrome('chromedriver.exe')
#first get the webpage
driver.get('https://www.thehindu.com/archive/')

In [89]:
jun_15=driver.find_element_by_xpath("//ul[7][@class='archiveMonthList']/li[6]/a")
driver.get(jun_15.get_attribute("href"))


In [90]:
page=requests.get('https://www.thehindu.com/archive/web/2015/06/')


In [91]:
date=[]
soup = BeautifulSoup(page.text, 'html.parser')
block = soup.find('table', class_='archiveTable')
link = block.find_all('a')
#Scrapping Date Link
for i in link:
    date.append(i.get('href'))

In [92]:
news_link=[]
#Scrapiing All news Link
for i in date:
    page=requests.get(i)
    soup = BeautifulSoup(page.text, 'html.parser')
    col = soup.find('div', class_='tpaper-container')
    sec = col.find_all('section')
    for i in sec:
        b = i.find_all('ul', class_='archive-list')
        for j in b:
            c = j.find_all('a')
            for k in c:
                news_link.append(k.get('href'))

In [111]:
verticals=[]
authors=[]
Headlines=[]
time=[]

In [117]:
for i in news_link:
    page=requests.get(i)
    soup = BeautifulSoup(page.text, 'html.parser')
    
    #Scrapping Verticals
    try:
        ver1 = soup.find('div', class_='article')
        ver2 = ver1.find('div', class_='article-exclusive').a.text.replace('\n','')
        verticals.append(ver2)
    except AttributeError:
        verticals.append('')
    
    #Scrapping Heading
    try:
        h1 = soup.find('div', class_='article').div.h1.text.replace('\n','')
        Headlines.append(h1)
    except AttributeError:
        Headlines.append('')
    
    #Scrapping Date of Upload
    try:
        d1 = soup.find('div', class_='article')
        d2 = d1.find('div', class_='ut-container')
        d3 = d2.find_all('span')
        d4 = d3[1].text.replace('\n','')
        time.append(d4)
    except AttributeError:
        time.append('')
    
    #Scrapping Author Name
    try:
        a1 = soup.find('div', class_='article')
        a2 = d1.find('div', class_='author-container hidden-xs').span.text.replace('\n','')
        authors.append(a2)
    except AttributeError:
        authors.append('')
     
 

In [99]:
for i in news_link:
    driver.get(i)
    timeout=12
    
    
    des = driver.find_elements_by_xpath("//div[@class='article']/div[3]")
    for i in des:
        description.append(i.text.replace('\n','').replace('\n\n','').replace('\n\n\n',''))

In [128]:
print(len(authors),len(verticals),len(Headlines),len(time),len(description))

16902 16902 16902 16902 16902


In [120]:
#creating dataframe
TheHindu=pd.DataFrame({})
TheHindu['Date_and_time_of_upload']=time
TheHindu['Author_name']=authors
TheHindu['Topic_or_Vertical']=verticals
TheHindu['Headlines']=Headlines
TheHindu['Description']=description
TheHindu

,Date_and_time_of_upload,Author_name,Topic_or_Vertical,Headlines,Description
0,"June 01, 2015 12:15 IST",PTI,Other Sports,Bhanot elected Asian Athletics Association vic...,He is currently the chairman of AFI’s Planning...
1,"June 01, 2015 02:17 IST",AP,Other Sports,Injured Lavillenie wins,Gold for Gatlin and GayKirani James won the 40...
2,"June 01, 2015 00:33 IST","BENGALURU:,",Races,Smile Stone bags feature event,Mr. K. Mallikarjuna Rao’s Smile Stone (Suraj N...
3,"June 01, 2015 00:32 IST","UDHAGAMANDALAM:,",Races,Panthera appeals most,"Panthera appeals most in the Vellore Plate (1,..."
4,"June 01, 2015 20:47 IST",AP,Tennis,Serena edges Sloane Stephens at French Open,Serena Williams came back to edge Sloane Steph...
...,...,...,...,...,...
16897,"July 03, 2015 03:11 IST",Lalithaa Krishnan,Friday Review,Art imitates ART,Self-taught artist Sajeesh Areeppurath’s fasci...
16898,"June 30, 2015 22:26 IST",Vidya Venkat,Environment,Green Climate Fund to become operational soon,The $100-billion Green Climate Fund will soon ...
16899,"June 30, 2015 21:55 IST","June 30, 2015 21:55 IST",Faith,The highest knowledge,The term Vedanta literally means the end of th...
16900,"July 02, 2015 18:26 IST",V. V. RAMANI,Music,Musical boundaries merged in Naada Yoga,Veterans shared the stage with youngsters and ...


In [122]:
#csv file
TheHindu.to_csv('the_hindu_june_2015.csv')

In [123]:
#saving as json file
import json

thehindu1=TheHindu.to_json('the_hindu_june_2015.json')

In [124]:
with open('the_hindu_june_2015.json','r') as file:
    t=json.load(file)

In [127]:
#saving as xml file
xml_data = ['<root>']
for column in TheHindu.columns:
    xml_data.append('<{}>'.format(column))                   
    for field in TheHindu.index:
        # writing sub-elements
        xml_data.append('<{0}>{1}</{0}>'.format(field, TheHindu[column][field]))
    xml_data.append('</{}>'.format(column))  
xml_data.append('</root>')

with open('thehindu.xml', 'w',encoding='utf-8') as f:       
    for line in xml_data:
        f.write(line)